<a href="https://colab.research.google.com/github/mahaairshad-shu/MechanicsPython2025/blob/main/Week%203/Wk3_Interactive_3D_plotting_and_slider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Interactive 3D plot using Plotly

##Example: Helix plot

In [ ]:
# 1. Install Plotly and other necessary libraries if running outside Colab
# (In Google Colab, plotly is usually pre-installed)
# !pip install plotly pandas

import plotly.express as px
import pandas as pd
import numpy as np

# --- 2. Create Sample Data (a Spiral/Helix Shape) ---
# Create 100 data points
num_points = 100
t = np.linspace(-15, 15, num_points)

# Define the coordinates for a helix
x_data = np.cos(t)
y_data = np.sin(t)
z_data = t

# Define a color/size attribute (e.g., based on the magnitude of t)
color_data = np.abs(t)

# Create a Pandas DataFrame for Plotly Express
df = pd.DataFrame({
    'X_Axis': x_data,
    'Y_Axis': y_data,
    'Z_Axis': z_data,
    'Color_Scale': color_data
})

# --- 3. Generate the 3D Scatter Plot ---
# px.scatter_3d is the function for 3D scatter plots
fig = px.scatter_3d(
    df,
    x='X_Axis',
    y='Y_Axis',
    z='Z_Axis',
    color='Color_Scale',     # Use the 'Color_Scale' column to color the points
    opacity=0.7,
    title='Interactive 3D Helix Plot'
)

# --- 4. Customize Layout (Optional) ---
# Enhance the visual appearance
fig.update_layout(
    scene = dict(
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        zaxis_title='Z Coordinate'
    ),
    margin=dict(l=0, r=0, b=0, t=40) # Adjust margins for better fit
)

# --- 5. Display the Plot ---
# In a Colab notebook, this will render the interactive plot
fig.show()

##Example: Ball trajectory plot

In [ ]:
import numpy as np
import plotly.graph_objects as go

# --- 1. Physics Constants and Setup ---
# Acceleration due to gravity (m/s^2)
G = 9.81

# Fixed launch angles (in degrees)
THETA_DEG = 45 # Elevation angle
PHI_DEG = 30   # Azimuth angle

# Convert angles to radians
THETA_RAD = np.deg2rad(THETA_DEG)
PHI_RAD = np.deg2rad(PHI_DEG)

# --- 2. Define the Speeds to Compare ---
# Four different initial launch speeds (V0) in m/s
LAUNCH_SPEEDS = [15.0, 25.0, 35.0, 45.0]
COLORS = ['#3a86ff', '#ff006e', '#8338ec', '#ffbe0b'] # A list of distinct colors

print(f"Simulation Setup:")
print(f"- Elevation Angle (Theta): {THETA_DEG}°")
print(f"- Azimuth Angle (Phi): {PHI_DEG}°")
print(f"- Launch Speeds: {LAUNCH_SPEEDS} m/s")
print("-" * 30)

# --- 3. Calculate Trajectories and Build Plotly Traces ---
data_traces = []
max_range = 0

for i, v0 in enumerate(LAUNCH_SPEEDS):
    # Calculate total time of flight (T_max = (2 * v0 * sin(theta)) / G)
    if v0 <= 0:
        t_max = 0
    else:
        t_max = (2 * v0 * np.sin(THETA_RAD)) / G

    # Generate time array (100 points for smooth curve)
    t = np.linspace(0, t_max, 100)

    # Calculate 3D position vectors
    # X: Range component
    x = v0 * np.cos(THETA_RAD) * np.cos(PHI_RAD) * t
    # Y: Cross-range component
    y = v0 * np.cos(THETA_RAD) * np.sin(PHI_RAD) * t
    # Z: Height component (parabolic motion)
    z = (v0 * np.sin(THETA_RAD) * t) - (0.5 * G * t**2)

    # Track the maximum range for better plot scaling
    if len(x) > 0:
        current_range = np.sqrt(x[-1]**2 + y[-1]**2)
        max_range = max(max_range, current_range)

    # Create the trace for the trajectory curve
    trajectory_trace = go.Scatter3d(
        x=x, y=y, z=z,
        mode='lines',
        line=dict(color=COLORS[i], width=5),
        name=f'V₀ = {v0:.1f} m/s'
    )
    data_traces.append(trajectory_trace)

    # Create a marker for the landing spot
    landing_trace = go.Scatter3d(
        x=[x[-1]] if len(x) > 0 else [0],
        y=[y[-1]] if len(y) > 0 else [0],
        z=[z[-1]] if len(z) > 0 else [0],
        mode='markers',
        marker=dict(size=8, color=COLORS[i]),
        showlegend=False # Don't show in legend, it's just the end marker
    )
    data_traces.append(landing_trace)


# --- 4. Create and Display the Final Figure ---
fig = go.Figure(data=data_traces)

# Set up the static layout
fig.update_layout(
    title='3D Projectile Trajectory Comparison at 4 Different Speeds',
    scene=dict(
        xaxis_title='Range (X-axis, meters)',
        yaxis_title='Cross-Range (Y-axis, meters)',
        zaxis_title='Height (Z-axis, meters)',

        # Ensure plot limits encompass all trajectories
        #xaxis=dict(range=[0, max_range * 1.1]),
        #yaxis=dict(range=[0, max_range * 1.1]),
        #zaxis=dict(range=[0, v0 * 20]), # Simple initial z-range estimate
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Display the static, interactive 3D plot
fig.show()

Simulation Setup:
- Elevation Angle (Theta): 45°
- Azimuth Angle (Phi): 30°
- Launch Speeds: [15.0, 25.0, 35.0, 45.0] m/s
------------------------------


#Interactive 3D plot with slider function

In [ ]:
import numpy as np
import plotly.graph_objects as go

# --- 1. Physics Constants and Setup ---
# Acceleration due to gravity (m/s^2)
G = 9.81

# Fixed launch angles (in degrees)
THETA_DEG = 45 # Elevation angle
PHI_DEG = 30   # Azimuth angle

# Convert angles to radians
THETA_RAD = np.deg2rad(THETA_DEG)
PHI_RAD = np.deg2rad(PHI_DEG)

# Discrete speed values to be used by the Slider
LAUNCH_SPEEDS = [15.0, 25.0, 35.0, 45.0]
INITIAL_SPEED = LAUNCH_SPEEDS[1] # Start with 25.0 m/s

print(f"Simulation Setup:")
print(f"- Elevation Angle (Theta): {THETA_DEG}°")
print(f"- Azimuth Angle (Phi): {PHI_DEG}°")
print(f"- Launch Speeds: {LAUNCH_SPEEDS} m/s")
print("-" * 30)


# --- 2. Trajectory Calculation Helper Function ---

def calculate_trajectory(v0):
    """Calculates x, y, z arrays for a given initial velocity v0."""
    if v0 <= 0:
        t_max = 0
    else:
        # T_max = (2 * v0 * sin(theta)) / G
        t_max = (2 * v0 * np.sin(THETA_RAD)) / G

    # Generate time array
    t = np.linspace(0, t_max, 100)

    # Calculate 3D position vectors
    x = v0 * np.cos(THETA_RAD) * np.cos(PHI_RAD) * t
    y = v0 * np.cos(THETA_RAD) * np.sin(PHI_RAD) * t
    z = (v0 * np.sin(THETA_RAD) * t) - (0.5 * G * t**2)

    return x, y, z

# --- 3. Initial Plot Setup and Calculation ---

# Calculate initial data
initial_x, initial_y, initial_z = calculate_trajectory(INITIAL_SPEED)

# Create the trajectory line trace (Trace 0)
trajectory_trace = go.Scatter3d(
    x=initial_x, y=initial_y, z=initial_z,
    mode='lines',
    line=dict(color='#3a86ff', width=5),
    name=f'Ball Trajectory (V₀={INITIAL_SPEED:.1f} m/s)'
)
# Create the landing spot marker trace (Trace 1)
landing_trace = go.Scatter3d(
    x=[initial_x[-1]], y=[initial_y[-1]], z=[initial_z[-1]],
    mode='markers',
    marker=dict(size=8, color='#ff006e'),
    name='Landing Spot'
)

# Initialize the figure
fig = go.Figure(data=[trajectory_trace, landing_trace])


# --- 4. Create Slider Steps ---

slider_steps = []
for i, v0 in enumerate(LAUNCH_SPEEDS):
    x_data, y_data, z_data = calculate_trajectory(v0)

    # Text label for the step on the slider (shows the speed)
    label = f'{v0:.1f} m/s'

    # Command to execute when the slider hits this step (restyle the data traces)
    step = dict(
        method='restyle',
        args=[
            # Data arguments (applied to all traces by default)
            {
                'x': [x_data, [x_data[-1]]],
                'y': [y_data, [y_data[-1]]],
                'z': [z_data, [z_data[-1]]],
                'name': [f'Ball Trajectory (V₀={v0:.1f} m/s)', 'Landing Spot']
            }
        ],
        label=label
    )
    slider_steps.append(step)

# --- 5. Configure Layout with Slider ---

fig.update_layout(
    title='Projectile Trajectory Simulator (Adjust Speed using Slider)',

    # Configuration for the native Plotly Slider
    sliders=[dict(
        active=LAUNCH_SPEEDS.index(INITIAL_SPEED), # Initial position
        currentvalue={"prefix": "Launch Speed: "},
        pad={"t": 50},
        steps=slider_steps,
        transition={'duration': 100, 'easing': 'cubic-in-out'},

        # --- RESIZING SLIDER ---
        len=0.5, # Slider length is 50% of the plot width
        x=0.25   # Start x-position at 25% to center it (0.25 + 0.5 = 0.75)
        # -----------------------------------
    )],

    scene=dict(
        xaxis_title='Range (X-axis, meters)',
        yaxis_title='Cross-Range (Y-axis, meters)',
        zaxis_title='Height (Z-axis, meters)',
        # Ensure initial ranges are set
        zaxis=dict(range=[0, 60]),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Display the final interactive figure
fig.show()

Simulation Setup:
- Elevation Angle (Theta): 45°
- Azimuth Angle (Phi): 30°
- Launch Speeds: [15.0, 25.0, 35.0, 45.0] m/s
------------------------------
